In [5]:
library('quantmod')
library('tidyquant')
library('rugarch')
library('copula')
library('VGAM')
library('ggplot2')
library('GoFKernel')
library('mistr')
library('tseries')
library('stats')
library('fDMA')
library('corrplot')

Loading required package: xts

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loading required package: lubridate


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Loading required package: PerformanceAnalytics


Attaching package: 'PerformanceAnalytics'


The following object is masked from 'package:graphics':

    legend


Loading required package: parallel


Attaching package: 'rugarch'


The following object is masked from 'package:stats':

    sigma



Attaching package: 'copula'


The following object is masked from 'package:lubridate':

    interval


Loading required package: stats4

Loading required package: splines


Attaching package: 'VGAM'


The following objects are maske

In [12]:
delta <- 0.5
clayton <- claytonCopula(delta,dim = 2)
rot_clayton <- rotCopula(clayton)
lambda(rot_clayton)
# cCopula(cbind(0.5,0.5),indices = 2,clayton)

lower upper 
 0.00  0.25

lower upper 
  0.0   0.5

In [50]:
## ground true
gt_clayton <- function(u,v){
    (u^(-delta)+v^{-delta}-1)^(-1 / delta)
}

gt_rot_clayton <- function(u,v) {
      u+v-1+gt_clayton(1-u,1-v)
    }

conditional_clayton <- function(u,v){
    (1 + u^delta * (v^(-delta) - 1))^(-1 / delta - 1)
}

In [51]:
print(conditional_clayton(0.5,0.5))

[1] 0.4444444


In [44]:
u1 <- 0.5  # 第一个变量的值
u2 <- 0.5  # 第二个变量的值
p1 <- pCopula(cbind(u1, u2), clayton)
p2 <- gt_clayton(u1, u2)
print(p1)
print(p2)

[1] 0.3333333
[1] 0.3333333


In [45]:
u1 <- 0.5  # 第一个变量的值
u2 <- 0.5  # 第二个变量的值
p1 <- pCopula(cbind(u1, u2), rot_clayton)
p2 <- gt_rot_clayton(u1, u2)
print(p1)
print(p2)

[1] 0.3333333
[1] 0.3333333


In [13]:
(Xtras <- copula:::doExtras()) # determine whether examples will be extra (long)
n <- if(Xtras) 200 else 64 # sample size
## A Gumbel copula
set.seed(7) # for reproducibility
gumbel.cop <- gumbelCopula(3, dim=2)
x <- rCopula(n, gumbel.cop) # "true" observations (simulated)
u <- pobs(x)                # pseudo-observations
## Inverting Kendall's tau
fit.tau <- fitCopula(gumbelCopula(), u, method="itau")
fit.tau
confint(fit.tau) # work fine !
confint(fit.tau, level = 0.98)
summary(fit.tau) # a bit more, notably "Std. Error"s
coef(fit.tau)# named vector
coef(fit.tau, SE = TRUE)# matrix

[1] FALSE

Call: fitCopula(gumbelCopula(), data = u, ... = pairlist(method = "itau"))
Fit based on "inversion of Kendall's tau" and 64 2-dimensional observations.
Copula: gumbelCopula 
alpha 
3.262 

,2.5 %,97.5 %
alpha,2.099906,4.424366


,1 %,99 %
alpha,1.882645,4.641627


Call: fitCopula(gumbelCopula(), data = u, ... = pairlist(method = "itau"))
Fit based on "inversion of Kendall's tau" and 64 2-dimensional observations.
Gumbel copula, dim. d = 2 
      Estimate Std. Error
alpha    3.262      0.593

alpha 
3.262136

,Estimate,Std. Error
alpha,3.262136,0.5929855


In [54]:
foo <- gumbelCopula(3.2,dim=2)

In [57]:
rho(foo)

[1] 0.8665436

In [ ]:
ugarchspec()

In [45]:
foo <- GNG_fit(stocks$SAP)

In [46]:
foo

Fitted composite GPD-Normal-GPD distribution: 

Breakpoints: -0.019414 0.019353 
Weights: 0.091343 0.812546 0.096112 

Parameters: 
    loc1   scale1   shape1     mean       sd     loc2   scale2   shape2 
0.019414 0.013439 0.150141 0.000907 0.011696 0.019353 0.010842 0.096832 

Log-likelihood: 7423.245,  Average log-likelihood: 2.7231


In [49]:
foo$params

$breakpoints
     break1      break2 
-0.01941394  0.01935279 

$weights
[1] 0.09134263 0.81254585 0.09611152

$coeff
       loc1      scale1      shape1        mean          sd        loc2 
0.019413938 0.013439092 0.150141199 0.000906912 0.011696163 0.019352790 
     scale2      shape2 
0.010841623 0.096832363

In [51]:
foo$Distribution

Composite distribution with: 
 
  Trafo       Distribution                                   Parameters Weight
1  -X   Generalized Pareto loc = 0.0194, scale = 0.0134, shape = 0.1501 0.0913
2 none        Normal                mean = 9e-04, sd = 0.0117           0.8125
3 none  Generalized Pareto loc = 0.0194, scale = 0.0108, shape = 0.0968 0.0961
        Truncation
1  (-Inf,-0.0194) 
2 [-0.0194,0.0194)
3   [0.0194,Inf)  

In [65]:
nlrq.control(maxiter = 1000,eps = 1e-04)

$maxiter
[1] 1000

$k
[1] 2

$InitialStepSize
[1] 1

$big
[1] 1e+20

$eps
[1] 1e-07

$beta
[1] 0.97